# Processamento de Linguagem Natural aplicada à Gestão Pública

Aula 5 (08/06): Reconhecimento de entidades nomeadas, Classificação de Textos e Modelagem de Tópicos

Reconhecimento de menção a pessoas, lugares e organizações e outras entidades. Classificação de Textos. Identificação de tópicos. Alocação Latente de Dirichlet.



## Preparação do ambiente

In [24]:
# Download dos dados
%%capture
!git clone https://github.com/samuelbarbosaa/oficina_nlp.git

In [25]:
import pickle
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

plt.rcParams['figure.figsize'] = [10, 5]

In [26]:
# Carrega textos já processados
with open('/content/oficina_nlp/data/textos_padronizados.pkl', 'rb') as fp:
  textos_proposicoes = pickle.load(fp)

textos_proposicoes

texto_padr  \
siglaTipoProjeto numero ano                                                       
PL               502    1999  projeto lei autorizar executivo criar concessã...   
                 2126   2002  projeto lei alterar dispositivo lei dezembro c...   
                 2127   2002  projeto lei alterar redação art lei dezembro d...   
                 120    2003  projeto lei projeto lei instituir medalha méri...   
                 1519   2004  projeto lei dispor gratuidade tranporte coleti...   
...                                                                         ...   
                 3614   2022  projeto lei autorizar executivo doar município...   
PEC              81     2022  proposta emenda constituição acrescentar art i...   
PL               3622   2022  projeto lei declarar utilidade público institu...   
                 3624   2022  projeto lei denominação viaduto localizar alça...   
                 3625   2022  projeto lei autorizar executivo doar município...   

                                                                          texto  
siglaTipoProjeto numero ano                                                      
PL               502    1999  PROJETO DE LEI Nº 502/99 Autoriza o Poder Exec...  
                 2126   2002  PROJETO DE LEI Nº 2.126/2002 Altera dispositiv...  
                 2127   2002  PROJETO DE LEI Nº 2.127/2002 Altera a redação ...  
                 120    2003  PROJETO DE LEI Nº 120/2003 (EX-PROJETO DE LEI ...  
                 1519   2004  PROJETO DE LEI Nº 1.519/2004 Dispõe sobre a gr...  
...                                                                         ...  
                 3614   2022  # Projeto de Lei nº 3.614/2022\n\nAutoriza o P...  
PEC              81     2022  # Proposta de Emenda à Constituição nº 81/2022...  
PL               3622   2022  # Projeto de Lei nº 3.622/2022\n\nDeclara de u...  
                 3624   2022  # Projeto de Lei nº 3.624/2022\n\nDá denominaç...  
                 3625   2022  # Projeto de Lei nº 3.625/2022\n\nAutoriza o P...  

[4534 rows x 2 columns]

Assuntos a que se referem cada proposição (*label*/classes):

In [27]:
assuntos = pd.read_csv("/content/oficina_nlp/data/assunto_proposicoes.csv").drop_duplicates().set_index(["siglaTipoProjeto", "numero", "ano"])
assuntos

assuntoGeral
siglaTipoProjeto numero ano                                                    
PEC              1      1972                            Administração Municipal
                        1972                              Constituição Estadual
                        1975  Assembleia Legislativa do Estado de Minas Gera...
                        1975                              Constituição Estadual
                        1975                                  Deputado Estadual
...                                                                         ...
PL               3776   2022                              Transporte e Trânsito
                 3778   2022                                Patrimônio Cultural
                 3779   2022                                             Evento
                 2383   2020                                           Educação
                 4861   2017                            Ensino Público Estadual

[64220 rows x 1 columns]

Utilizaremos como variáveis preditoras (*features*) a matriz LSA.

Matriz TF-IDF:

In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer

metodo = TfidfVectorizer()
modelo = metodo.fit(textos_proposicoes["texto_padr"])
TF_IDF = modelo.transform(textos_proposicoes["texto_padr"])
TF_IDF

<4534x23841 sparse matrix of type '<class 'numpy.float64'>'
	with 729992 stored elements in Compressed Sparse Row format>

Matriz LSA

In [29]:
from sklearn.decomposition import TruncatedSVD

tsvd = TruncatedSVD(n_components=10)
M_LSA = tsvd.fit_transform(TF_IDF)

In [30]:
M_LSA.shape

(4534, 10)

## Construção da matriz $X$, de variáveis preditoras

In [31]:
X = pd.DataFrame(M_LSA, index=textos_proposicoes.index)
X

0         1         2         3  \
siglaTipoProjeto numero ano                                            
PL               502    1999  0.197148 -0.070334  0.006180  0.019315   
                 2126   2002  0.219005 -0.116131 -0.043454  0.021444   
                 2127   2002  0.291456 -0.146212 -0.025173  0.046497   
                 120    2003  0.147777 -0.015505 -0.011531 -0.023840   
                 1519   2004  0.260409 -0.120940 -0.012898 -0.050121   
...                                ...       ...       ...       ...   
                 3614   2022  0.294968  0.036816  0.351056 -0.014532   
PEC              81     2022  0.260494 -0.059573 -0.031587 -0.046052   
PL               3622   2022  0.277894  0.212062 -0.073688 -0.005789   
                 3624   2022  0.108521  0.021439  0.000908  0.001291   
                 3625   2022  0.304978  0.039005  0.400510 -0.000823   

                                     4         5         6         7  \
siglaTipoProjeto numero ano                                            
PL               502    1999 -0.007822 -0.055337  0.009857  0.040396   
                 2126   2002 -0.030288 -0.184952 -0.005094  0.107220   
                 2127   2002 -0.045563 -0.253030 -0.006564  0.150605   
                 120    2003 -0.023328 -0.003911 -0.020818  0.022958   
                 1519   2004 -0.068699 -0.028392 -0.008942  0.151119   
...                                ...       ...       ...       ...   
                 3614   2022  0.008926  0.066146  0.047059 -0.022350   
PEC              81     2022 -0.095933 -0.003337 -0.064729 -0.004338   
PL               3622   2022 -0.024179  0.035966  0.010046 -0.003185   
                 3624   2022 -0.000109 -0.015583  0.023233  0.041107   
                 3625   2022  0.008378  0.037337  0.030407 -0.023105   

                                     8         9  
siglaTipoProjeto numero ano                       
PL               502    1999 -0.044711  0.034256  
                 2126   2002  0.101947 -0.071822  
                 2127   2002  0.136953 -0.102723  
                 120    2003 -0.017735  0.005108  
                 1519   2004  0.056900 -0.138414  
...                                ...       ...  
                 3614   2022  0.025295  0.017744  
PEC              81     2022 -0.071300  0.024842  
PL               3622   2022  0.006937 -0.008978  
                 3624   2022  0.008079  0.026790  
                 3625   2022  0.000270  0.009775  

[4534 rows x 10 columns]

In [32]:
dados_classificacao = X.join(assuntos, how="inner")
dados_classificacao

0         1         2         3  \
siglaTipoProjeto numero ano                                            
PEC              1      2019  0.187110 -0.041069 -0.027076 -0.022069   
                        2019  0.187110 -0.041069 -0.027076 -0.022069   
                        2019  0.187110 -0.041069 -0.027076 -0.022069   
                        2019  0.187110 -0.041069 -0.027076 -0.022069   
                 2      2019  0.188398 -0.058256 -0.014387 -0.002127   
...                                ...       ...       ...       ...   
PRE              167    2022  0.219018 -0.063685  0.007136  0.311005   
                 171    2022  0.215951 -0.055345 -0.001155  0.318780   
                        2022  0.215951 -0.055345 -0.001155  0.318780   
                        2022  0.215951 -0.055345 -0.001155  0.318780   
                        2022  0.215951 -0.055345 -0.001155  0.318780   

                                     4         5         6         7  \
siglaTipoProjeto numero ano                                            
PEC              1      2019 -0.104861 -0.064377 -0.194709 -0.033675   
                        2019 -0.104861 -0.064377 -0.194709 -0.033675   
                        2019 -0.104861 -0.064377 -0.194709 -0.033675   
                        2019 -0.104861 -0.064377 -0.194709 -0.033675   
                 2      2019 -0.088998 -0.062617 -0.175627 -0.038617   
...                                ...       ...       ...       ...   
PRE              167    2022  0.070929  0.053633 -0.206210  0.085730   
                 171    2022  0.079152  0.060715 -0.194591  0.083473   
                        2022  0.079152  0.060715 -0.194591  0.083473   
                        2022  0.079152  0.060715 -0.194591  0.083473   
                        2022  0.079152  0.060715 -0.194591  0.083473   

                                     8         9  \
siglaTipoProjeto numero ano                        
PEC              1      2019 -0.094415  0.017330   
                        2019 -0.094415  0.017330   
                        2019 -0.094415  0.017330   
                        2019 -0.094415  0.017330   
                 2      2019 -0.081991  0.015682   
...                                ...       ...   
PRE              167    2022 -0.134410 -0.024340   
                 171    2022 -0.144922 -0.021876   
                        2022 -0.144922 -0.021876   
                        2022 -0.144922 -0.021876   
                        2022 -0.144922 -0.021876   

                                                       assuntoGeral  
siglaTipoProjeto numero ano                                          
PEC              1      2019                 Administração Estadual  
                        2019                  Constituição Estadual  
                        2019                                Pessoal  
                        2019                        Pessoal Militar  
                 2      2019                  Constituição Estadual  
...                                                             ...  
PRE              167    2022                          Saúde Pública  
                 171    2022                     Calamidade Pública  
                        2022                      Finanças Públicas  
                        2022  Municípios e Desenvolvimento Regional  
                        2022                          Saúde Pública  

[10221 rows x 11 columns]

In [33]:
X = dados_classificacao.drop(columns=["assuntoGeral"])
Y = dados_classificacao[["assuntoGeral"]]

Separação dos dados em conjunto de treinamento e de teste:

In [34]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

## Método 01: *Random Forest*

In [35]:
from sklearn.ensemble import RandomForestClassifier

metodo = RandomForestClassifier(n_estimators=200, random_state=0)
modelo = metodo.fit(X_train, Y_train) 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.


In [36]:
Y_predito = modelo.predict(X_test)
Y_predito

array(['Crédito', 'Estabelecimento Penal', 'Conselho Estadual', ...,
       'Pessoal', 'Defesa do Consumidor', 'Segurança Pública'],
      dtype=object)

In [37]:
print(Y_predito.shape, Y_test.shape)

(2045,) (2045, 1)


### Avaliando o modelo

In [38]:
from sklearn.metrics import classification_report, accuracy_score

accuracy_score(Y_train, modelo.predict(X_train))

0.5047700587084148

In [39]:
print(f"Acurácia (geral) do modelo: {accuracy_score(Y_test, Y_predito)}")
print("* Qual o percentual de vezes que o modelo prediz corretamente a categoria.")

Acurácia (geral) do modelo: 0.16674816625916872
* Qual o percentual de vezes que o modelo prediz corretamente a categoria.


Relatório por Categoria:

*   Precisão: percentual de acerto na categoria [TP / (TP+FP)]
*   Recall: percentual de vezes que o modelo foi capaz de predizer a categoria [TP / (TP+FN)]

In [40]:
print(classification_report(Y_test, Y_predito))

                                                                          precision    recall  f1-score   support

                                                                  (LGBT)       0.00      0.00      0.00         4
                                                       Acordo Financeiro       0.00      0.00      0.00         1
                                                  Administração Estadual       0.02      0.03      0.02        40
                                                 Administração Municipal       0.00      0.00      0.00         4
                                                   Administração Pública       0.07      0.06      0.06        16
                                                            Agropecuária       0.00      0.00      0.00        22
                                                             Alimentação       0.00      0.00      0.00        18
                                                              Artesanato       0.00    

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1

In [41]:
from sklearn.linear_model import LogisticRegression

In [42]:
clf_LR = LogisticRegression(multi_class="multinomial")

In [43]:
fit_LR = clf_LR.fit(X_train, Y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [44]:
Y_pred_LR = fit_LR.predict(X_test)

In [45]:
print(f"Acurácia (geral) do modelo: {accuracy_score(Y_test, Y_pred_LR)}")
print("* Qual o percentual de vezes que o modelo prediz corretamente a categoria.")

Acurácia (geral) do modelo: 0.2723716381418093
* Qual o percentual de vezes que o modelo prediz corretamente a categoria.


In [46]:
print(classification_report(Y_test, Y_pred_LR))

                                                                          precision    recall  f1-score   support

                                                                  (LGBT)       0.00      0.00      0.00         4
                                                       Acordo Financeiro       0.00      0.00      0.00         1
                                                  Administração Estadual       0.19      0.12      0.15        40
                                                 Administração Municipal       0.00      0.00      0.00         4
                                                   Administração Pública       0.00      0.00      0.00        16
                                                            Agropecuária       0.00      0.00      0.00        22
                                                             Alimentação       0.00      0.00      0.00        18
                                                              Artesanato       0.00    

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
